In [236]:
using Flux
using Flux: argmax, crossentropy, throttle
using Base.Iterators: repeated
using JLD
using LinearAlgebra

In [237]:
X = load("C:/Users/kgtrm/Documents/VSC Code/julia stuff/tictactoe/dataX.jld")["data"]
X = copy(X')
Y = load("C:/Users/kgtrm/Documents/VSC Code/julia stuff/tictactoe/dataY.jld")["data"]
Y = copy(Y')
;

In [238]:
neural_net = Chain(Dense(9, 36, relu), Dense(36, 36, relu), Dense(36, 9), softmax)

Chain(Dense(9, 36, relu), Dense(36, 36, relu), Dense(36, 9), softmax)

In [239]:
loss(x,y) = Flux.crossentropy(neural_net(x),y)
accuracy(x,y) = mean(argmax(neural_net(x)) .== argmax(y))

trainids = unique(rand(1:size(X)[2], 1650))
trainingX = X[:, trainids]
trainingY = Y[:, trainids]
testids = setdiff(1:size(X)[2], trainids)
testingX = X[:, testids]
testingY = Y[:, testids]
;

datasetx = repeated((trainingX,trainingY),200)
C = collect(datasetx);

evalcb = () -> @show(loss(X,Y))

ps = Flux.params(neural_net)

opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any, Any}())

In [249]:
Flux.train!(loss, ps, datasetx, opt, cb = throttle(evalcb, 10))

loss(X, Y) = 0.032663984774585895


In [250]:
correct = 0
total = length(testids)
for i in 1:length(testids)
    if argmax(neural_net(testingX[:,i])) == argmax(testingY[:,i])
        correct += 1
    end
end

percent_correct = correct / total * 100

98.17549956559513

In [251]:
function generate_blank_board()
    board = []
    for i in 1:9
        push!(board, 0)
    end
    return board
end

function display_board(board)
    for i in 1:9
        if (board[i] == -1)
            print("|O")
        elseif (board[i] == 1)
            print("|X")
        elseif (board[i] == 0)
            print("|_")
        end
        if (i % 3 == 0)
            print("\n")
        end
    end
end

function is_X_win(board)
    if (board[3] == board[5] == board[7] == 1)
        return true
    elseif (board[1] == board[2] == board[3] == 1)
        return true
    elseif (board[4] == board[5] == board[6] == 1)
        return true
    elseif (board[7] == board[8] == board[9] == 1)
        return true
    elseif (board[1] == board[4] == board[7] == 1)
        return true
    elseif (board[2] == board[5] == board[8] == 1)
        return true
    elseif (board[3] == board[6] == board[9] == 1)
        return true
    elseif (board[1] == board[5] == board[9] == 1)
        return true
    end
    return false
end

function is_O_win(board)
    if (board[3] == board[5] == board[7] == -1)
        return true
    elseif (board[1] == board[2] == board[3] == -1)
        return true
    elseif (board[4] == board[5] == board[6] == -1)
        return true
    elseif (board[7] == board[8] == board[9] == -1)
        return true
    elseif (board[1] == board[4] == board[7] == -1)
        return true
    elseif (board[2] == board[5] == board[8] == -1)
        return true
    elseif (board[3] == board[6] == board[9] == -1)
        return true
    elseif (board[1] == board[5] == board[9] == -1)
        return true
    end
    return false
end

function full_board(board)
    full = true
    for i in board
        if i == 0
            full = false
        end
    end
    return full
end

function X_move!(board)
    position = argmax(neural_net(board))
    if board[position] == 0
        board[position] = 1
    else
        available_moves = []
        for place in 1:length(board)
            if board[place] == 0
                push!(available_moves, place)
            end
        end
        position = rand(available_moves)
        board[position] = 1
    end
end

function O_move!(board)
    available_moves = []
    for place in 1:length(board)
        if board[place] == 0
            push!(available_moves, place)
        end
    end
    position = rand(available_moves)
    board[position] = -1
end

function play_game()
    board = generate_blank_board()
    xturn = true
    while !is_O_win(board) && !is_X_win(board) && !full_board(board)
        # display_board(board)
        # println()
        if xturn
            X_move!(board)
            xturn = false
        else
            O_move!(board)
            xturn = true
        end
    end
    # display_board(board)
    # println("############")
    if is_O_win(board)
        return -1, board
    elseif is_X_win(board)
        return 1, board
    elseif full_board(board)
        return 0, board
    else
        return 2, board
    end
end

play_game (generic function with 1 method)

In [252]:
num_games = 1000000
num_wins = 0
num_cat = 0
num_loss = 0
count = 1

while count <= num_games
    outcome, final_board = play_game()
    if outcome == 1
        num_wins += 1
    elseif outcome == 0
        num_cat += 1
        # display_board(final_board)
        # println()
    elseif outcome == -1
        num_loss += 1
        # display_board(final_board)
        # println()
    else
        println("rogue board")
        # display_board(final_board)
        # println()
    end
    count += 1
end

println("number of wins: ", num_wins, " win percentage: ", num_wins / num_games * 100)
println("number of losses: ", num_loss, " loss percentage: ", num_loss / num_games * 100)
println("number of cats: ", num_cat, " cat percentage: ", num_cat / num_games * 100)

number of wins: 952575 win percentage: 95.2575
number of losses: 34244 loss percentage: 3.4243999999999994
number of cats: 13181 cat percentage: 1.3181
